# Simple Strategy Backtest

Backtest a simple moving average crossover strategy on historical BTCUSDT data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ccxt
from datetime import datetime, timedelta

# Initialize Binance exchange
exchange = ccxt.binance()

# Fetch OHLCV data
symbol = 'BTC/USDT'
timeframe = '1h'
limit = 100

ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

print(f'Data shape: {df.shape}')
print(df.head())

In [ ]:
# Simple Moving Average Strategy
def calculate_sma(df, fast=10, slow=20):
    df['sma_fast'] = df['close'].rolling(window=fast).mean()
    df['sma_slow'] = df['close'].rolling(window=slow).mean()
    return df

# Add indicators
df = calculate_sma(df, fast=10, slow=20)

# Generate signals
df['signal'] = 0
df.loc[df['sma_fast'] > df['sma_slow'], 'signal'] = 1  # Buy
df.loc[df['sma_fast'] < df['sma_slow'], 'signal'] = -1  # Sell

# Position (actual position - changes in signal)
df['position'] = df['signal'].diff()

print(df[['close', 'sma_fast', 'sma_slow', 'signal', 'position']].tail(20))

In [ ]:
# Calculate returns
df['returns'] = df['close'].pct_change()
df['strategy_returns'] = df['signal'].shift(1) * df['returns']
df['cumulative_returns'] = (1 + df['returns']).cumprod()
df['cumulative_strategy'] = (1 + df['strategy_returns']).cumprod()

# Performance metrics
total_return = (df['cumulative_returns'].iloc[-1] - 1) * 100
strategy_return = (df['cumulative_strategy'].iloc[-1] - 1) * 100

print(f'Buy & Hold Return: {total_return:.2f}%')
print(f'Strategy Return: {strategy_return:.2f}%')

In [ ]:
# Visualization
plt.figure(figsize=(14, 7))

plt.subplot(2, 1, 1)
plt.plot(df['timestamp'], df['close'], label='Close Price', color='black')
plt.plot(df['timestamp'], df['sma_fast'], label='Fast SMA', color='blue', alpha=0.7)
plt.plot(df['timestamp'], df['sma_slow'], label='Slow SMA', color='red', alpha=0.7)
plt.fill_between(df['timestamp'], df['sma_fast'], df['sma_slow'], 
                 where=(df['sma_fast'] > df['sma_slow']), color='green', alpha=0.1)
plt.fill_between(df['timestamp'], df['sma_fast'], df['sma_slow'], 
                 where=(df['sma_fast'] <= df['sma_slow']), color='red', alpha=0.1)
plt.legend()
plt.title('BTC/USDT - SMA Crossover Strategy')
plt.ylabel('Price (USDT)')

plt.subplot(2, 1, 2)
plt.plot(df['timestamp'], df['cumulative_returns'], label='Buy & Hold', color='blue')
plt.plot(df['timestamp'], df['cumulative_strategy'], label='Strategy', color='green')
plt.legend()
plt.title('Cumulative Returns')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.tight_layout()
plt.show()

## Observations

The SMA crossover strategy shows how moving averages can be used to identify trends. Experiment with different parameters to optimize returns.